In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
np.random.seed(42)

In [2]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [3]:
df_train = pd.read_csv("output_files/combined.csv", index_col=0)

In [4]:
df_train.head()

ART_CODE                              DESC_CODED  \
0         0                           Hyponatraemia   
1         1  Subacute cutaneous lupus erythematosus   
2         2  Blood bilirubin unconjugated increased   
3         3              toxic epidermal necrolysis   
4         4                             Bradycardia   

                            HLGT_NAME_COMPL                 HLT_NAME_COMPL  \
0  ELECTROLYTE AND FLUID BALANCE CONDITIONS               SODIUM IMBALANCE   
1           EPIDERMAL AND DERMAL CONDITIONS    CONNECTIVE TISSUE DISORDERS   
2              HEPATOBILIARY INVESTIGATIONS        LIVER FUNCTION ANALYSES   
3           EPIDERMAL AND DERMAL CONDITIONS             BULLOUS CONDITIONS   
4                       CARDIAC ARRHYTHMIAS  RATE AND RHYTHM DISORDERS NEC   

     INC_CODE  INC_CODE_J                          LLT_NAME_COMPL  \
0  10021038.0    10021036                            HYPONATREMIA   
1  10057903.0    10057903  SUBACUTE CUTANEOUS LUPUS ERYTHEMATOSUS   
2  10021709.0    10021709            INDIRECT BILIRUBIN INCREASED   
3  10044223.0    10044223              TOXIC EPIDERMAL NECROLYSIS   
4  10006093.0    10006093                             BRADYCARDIA   

                            PT_NAME_COMPL  \
0                           HYPONATRAEMIA   
1  SUBACUTE CUTANEOUS LUPUS ERYTHEMATOSUS   
2  BLOOD BILIRUBIN UNCONJUGATED INCREASED   
3              TOXIC EPIDERMAL NECROLYSIS   
4                             BRADYCARDIA   

                                       REPORTED_TERM    SOC_CODE  len  
0                                       HYPONATREMIA  10027433.0    1  
1  OMEPRAZOLE INDUCED SUBACUTE CUTANEOUS LUPUS ER...  10040785.0    4  
2               INDIRECT BILIRUBIN (74.7 MICROMOL/L)  10022891.0    4  
3                         TOXIC EPIDERMAL NECROLYSIS  10040785.0    3  
4                                        BRADYCARDIA  10007541.0    1

In [5]:
MAX_WORDS = 26000
MAX_LEN = 40
EMBEDDING_DIM = 100

In [6]:
tokenizer = Tokenizer(num_words=MAX_WORDS, lower=True)

In [7]:
tokenizer.fit_on_texts(df_train.REPORTED_TERM)
sequences = tokenizer.texts_to_sequences(df_train.REPORTED_TERM)
x = pad_sequences(sequences, maxlen=MAX_LEN)

In [8]:
y = df_train.ART_CODE

In [9]:
dummies = pd.get_dummies(y.values)

In [10]:
y = np.array(dummies)

In [12]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [16]:
NUM_CLASSES = y.shape[1]

In [14]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalMaxPool1D, Dropout, SpatialDropout1D, LSTM
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

In [17]:
model = Sequential()
model.add(Embedding(MAX_WORDS, EMBEDDING_DIM, input_length=x.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(NUM_CLASSES, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

epochs = 30
batch_size = 64

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [18]:
model.fit(
    x_train, y_train, 
    epochs=epochs, batch_size=batch_size,
    validation_data=[x_test, y_test],
    callbacks=[
        ReduceLROnPlateau(),
        EarlyStopping(monitor='val_loss', patience=10, min_delta=0.0001),
        ModelCheckpoint(filepath='model-LSTM.h5', save_best_only=True)])

Instructions for updating:
Use tf.cast instead.
Train on 50586 samples, validate on 12647 samples
Epoch 1/30
50586/50586 [==============================] - 97s 2ms/step - loss: 6.8541 - categorical_accuracy: 0.0429 - val_loss: 6.3248 - val_categorical_accuracy: 0.0682
Epoch 2/30
50586/50586 [==============================] - 93s 2ms/step - loss: 5.8198 - categorical_accuracy: 0.1045 - val_loss: 5.4788 - val_categorical_accuracy: 0.1670
Epoch 3/30
50586/50586 [==============================] - 96s 2ms/step - loss: 4.8265 - categorical_accuracy: 0.2402 - val_loss: 4.5971 - val_categorical_accuracy: 0.3251
Epoch 4/30
50586/50586 [==============================] - 95s 2ms/step - loss: 3.9282 - categorical_accuracy: 0.3812 - val_loss: 3.9419 - val_categorical_accuracy: 0.4394
Epoch 5/30
50586/50586 [==============================] - 94s 2ms/step - loss: 3.2451 - categorical_accuracy: 0.4794 - val_loss: 3.4952 - val_categorical_accuracy: 0.5148
Epoch 6/30
50586/50586 [=======================

In [22]:
def predict(reported_term):
    #tokenizer.fit_on_texts(reported_term)
    sequences = tokenizer.texts_to_sequences(reported_term)
    x = pad_sequences(sequences, maxlen=MAX_LEN)
    return x

In [29]:
input_text = predict(pd.Series("INSOMNIA"))

In [31]:
ypred = model.predict(input_text)

In [32]:
np.argmax(ypred)

283

In [33]:
df_train[df_train.ART_CODE == 283]

ART_CODE            DESC_CODED                   HLGT_NAME_COMPL  \
419         283              Insomnia  SLEEP DISORDERS AND DISTURBANCES   
619         283         Sleeplessness  SLEEP DISORDERS AND DISTURBANCES   
1047        283              Insomnia  SLEEP DISORDERS AND DISTURBANCES   
1302        283              Insomnia  SLEEP DISORDERS AND DISTURBANCES   
1482        283              insomnia  SLEEP DISORDERS AND DISTURBANCES   
1789        283   Difficulty sleeping  SLEEP DISORDERS AND DISTURBANCES   
1844        283              Insomnia  SLEEP DISORDERS AND DISTURBANCES   
3428        283              insomnia  SLEEP DISORDERS AND DISTURBANCES   
4160        283              Insomnia  SLEEP DISORDERS AND DISTURBANCES   
5867        283              Insomnia  SLEEP DISORDERS AND DISTURBANCES   
6217        283              Insomnia  SLEEP DISORDERS AND DISTURBANCES   
7052        283              Insomnia  SLEEP DISORDERS AND DISTURBANCES   
7202        283              insomnia  SLEEP DISORDERS AND DISTURBANCES   
8601        283              Insomnia  SLEEP DISORDERS AND DISTURBANCES   
8959        283   Difficulty sleeping  SLEEP DISORDERS AND DISTURBANCES   
9012        283              Insomnia  SLEEP DISORDERS AND DISTURBANCES   
9464        283              Insomnia  SLEEP DISORDERS AND DISTURBANCES   
10425       283              Insomnia  SLEEP DISORDERS AND DISTURBANCES   
10579       283              Insomnia  SLEEP DISORDERS AND DISTURBANCES   
10684       283              Insomnia  SLEEP DISORDERS AND DISTURBANCES   
11879       283              insomnia  SLEEP DISORDERS AND DISTURBANCES   
12529       283              insomnia  SLEEP DISORDERS AND DISTURBANCES   
12737       283  insomnia exacerbated  SLEEP DISORDERS AND DISTURBANCES   
13289       283              Insomnia  SLEEP DISORDERS AND DISTURBANCES   
13869       283              Insomnia  SLEEP DISORDERS AND DISTURBANCES   
14275       283       sleep decreased  SLEEP DISORDERS AND DISTURBANCES   
14999       283              insomnia  SLEEP DISORDERS AND DISTURBANCES   
15658       283              Insomnia  SLEEP DISORDERS AND DISTURBANCES   
15718       283              Insomnia  SLEEP DISORDERS AND DISTURBANCES   
17344       283       sleep decreased  SLEEP DISORDERS AND DISTURBANCES   
...         ...                   ...                               ...   
913         283       Sleep decreased  SLEEP DISORDERS AND DISTURBANCES   
2005        283              Insomnia  SLEEP DISORDERS AND DISTURBANCES   
4882        283              Insomnia  SLEEP DISORDERS AND DISTURBANCES   
5070        283              insomnia  SLEEP DISORDERS AND DISTURBANCES   
5517        283              Insomnia  SLEEP DISORDERS AND DISTURBANCES   
6227        283              insomnia  SLEEP DISORDERS AND DISTURBANCES   
7336        283   difficulty sleeping  SLEEP DISORDERS AND DISTURBANCES   
8057        283              Insomnia  SLEEP DISORDERS AND DISTURBANCES   
8070        283              insomnia  SLEEP DISORDERS AND DISTURBANCES   
8302        283              Insomnia  SLEEP DISORDERS AND DISTURBANCES   
8861        283              insomnia  SLEEP DISORDERS AND DISTURBANCES   
9667        283              Insomnia  SLEEP DISORDERS AND DISTURBANCES   
10437       283       Sleep decreased  SLEEP DISORDERS AND DISTURBANCES   
10529       283              Insomnia  SLEEP DISORDERS AND DISTURBANCES   
11495       283         Sleeplessness  SLEEP DISORDERS AND DISTURBANCES   
12092       283              Insomnia  SLEEP DISORDERS AND DISTURBANCES   
12094       283              Insomnia  SLEEP DISORDERS AND DISTURBANCES   
12474       283              Insomnia  SLEEP DISORDERS AND DISTURBANCES   
12683       283              Insomnia  SLEEP DISORDERS AND DISTURBANCES   
12999       283              Insomnia  SLEEP DISORDERS AND DISTURBANCES   
13168       283         Sleeplessness  SLEEP DISORDERS AND DISTURBANCES   
14579       283              Ins